In [12]:
import torch
from torch import nn
from evotorch import Problem
from evotorch.logging import StdOutLogger
from evotorch.algorithms import SNES
from evotorch.algorithms import CEM
from evotorch.algorithms import GeneticAlgorithm
from evotorch.algorithms import XNES
from evotorch.algorithms import GeneticAlgorithm
from evotorch.decorators import vectorized
from evotorch.operators import TwoPointCrossOver
from evotorch.operators import OnePointCrossOver, GaussianMutation
from evotorch.operators import SimulatedBinaryCrossOver
from evotorch.operators import PolynomialMutation
import numpy as np
import random
import json
import pandas as pd
import numpy as np
import plotly.express as px



In [13]:

#import data from text file rt30.txt
rt30 = pd.read_csv('./example_data/rt30.txt', skiprows=36,error_bad_lines=False, sep=',',names=['Packet','intensity','mass/position','saturated','fragmented','merged'])
#pd.DataFrame(columns=['Packet','intensity','mass/position','saturated','fragmented','merged'])
#Store the data from the odd rows in the NaN values from the even rows
for i in range(0,len(rt30),2):
        rt30.loc[i,'saturated'] = rt30.loc[i+1,'Packet']
        rt30.loc[i,'fragmented'] = rt30.loc[i+1,'intensity']
        rt30.loc[i,'merged'] = rt30.loc[i+1,'mass/position']
#drop NaN rows
rt30 = rt30.dropna()
#remove all letters \ and = from the mass/position column and keep only numbers
#remove all non-numbers except for . from the mass/position column and convert to float
rt30['mass/position'] = rt30['mass/position'].str.replace('[^0-9.]', '')
#repeat for all other columns
rt30['Packet'] = rt30['Packet'].str.replace('[^0-9.]', '')
rt30['intensity'] = rt30['intensity'].str.replace('[^0-9.]','0')
rt30['saturated'] = rt30['saturated'].str.replace('[^0-9.]', '')
rt30['fragmented'] = rt30['fragmented'].str.replace('[^0-9.]', '')
rt30['merged'] = rt30['merged'].str.replace('[^0-9.]', '')
#convert all columns to float
rt30['mass/position'] = rt30['mass/position'].astype(float)
# rt30['Packet'] = rt30['Packet'].astype(float)
#remove spaces crom intensity column
rt30['intensity'] = rt30['intensity'].str.replace(' ', '')
rt30['intensity'] = rt30['intensity'].astype(float)
# rt30['saturated'] = rt30['saturated'].astype(float)
# rt30['fragmented'] = rt30['fragmented'].astype(float)
# rt30['merged'] = rt30['merged'].astype(float)




#drop the odd rows
#in pandas dataframe concat even and odd rows
#drop columns with NaN values
#rt30 = pd.concat([rt30.iloc[::2], rt30.iloc[1::2].reset_index(drop=True)], axis=1)
#drop columns with NaN values


#print type of data for column mass/position

print(rt30)
rt30.head()


/tmp/ipykernel_737/233339404.py:2: FutureWarning:

The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.



/tmp/ipykernel_737/233339404.py:13: FutureWarning:

The default value of regex will change from True to False in a future version.

/tmp/ipykernel_737/233339404.py:15: FutureWarning:

The default value of regex will change from True to False in a future version.

/tmp/ipykernel_737/233339404.py:16: FutureWarning:

The default value of regex will change from True to False in a future version.

/tmp/ipykernel_737/233339404.py:17: FutureWarning:

The default value of regex will change from True to False in a future version.

/tmp/ipykernel_737/233339404.py:18: FutureWarning:

The default value of regex will change from True to False in a future version.



      Packet  intensity  mass/position saturated fragmented merged
0          0   0.000000      50.083333         0          0      0
2          1   0.000000      50.166667         0          0      0
4          2   0.000000      50.250000         0          0      0
6          3   0.000000      50.333333         0          0      0
8          4   0.000000      50.416667         0          0      0
...      ...        ...            ...       ...        ...    ...
72256   1795   0.000000     199.666671         0          0      0
72258   1796   0.041165     199.750004         0          0      0
72260   1797   0.073314     199.833338         0          0      0
72262   1798   0.024089     199.916671         0          0      0
72264   1799   0.002376     200.000004         0          0      0

[36038 rows x 6 columns]


/tmp/ipykernel_737/233339404.py:19: FutureWarning:

The default value of regex will change from True to False in a future version.



,Packet,intensity,mass/position,saturated,fragmented,merged
0,0,0.0,50.083333,0,0,0
2,1,0.0,50.166667,0,0,0
4,2,0.0,50.250000,0,0,0
6,3,0.0,50.333333,0,0,0
8,4,0.0,50.416667,0,0,0


In [42]:
#sum up the intensity values for mass/position 96
df_sum = rt30
#round mass/position values to 0 decimals
df_sum['mass/position'] = df_sum['mass/position'].round(0)
#group by mass/position and sum up the intensity values
df_sum = df_sum.groupby('mass/position')['intensity'].sum().reset_index()




print(df_sum.nlargest(10, 'intensity'))
df_sum.loc[df_sum['intensity'].idxmax()]

     mass/position     intensity
47            96.0  1.070786e+07
119          168.0  8.396774e+05
118          167.0  1.274593e+05
64           113.0  4.856332e+04
120          169.0  3.673823e+04
46            95.0  3.397643e+04
48            97.0  2.282794e+04
24            73.0  1.933824e+04
117          166.0  1.246411e+04
25            74.0  6.359322e+03


mass/position    9.600000e+01
intensity        1.070786e+07
Name: 47, dtype: float64

In [49]:
#plot mass/position vs intensity in plotly in a scatter plot
#fig = px.line(rt30, x="mass/position", y="intensity", title='Orbitrap Data')
fig = px.bar(rt30, x="mass/position", y="intensity", title='Orbitrap Data')
fig.show()
fig1 = px.bar(df_sum, x="mass/position", y="intensity", title='Orbitrap Data')
fig1.show()

In [18]:
# #prompt for output
def prompt(parameters: torch.Tensor) -> torch.Tensor:
    print(parameters.shape)
    #prompt user for input from jupyter notebook
    print('1-Collision Energy, 2-Retention Time')
    #for loop to print out parameters their representive string 
    for i in range(len(parameters)):
        if i == 0:
            print('Collision Energy  = {}'.format(parameters[i]))
        if i == 1:
            print('Retention Time = {}'.format(parameters[i]))
    #print('Please run instrument with parameters {}'.format(parameters))
    return torch.tensor(float(input("Enter a number: ")))


# def objective(parameters: torch.Tensor) -> torch.Tensor:
    


#1 Collision Energy = 0 - 50 ??
#2 Retention Time = 30 - 500 ms?


In [24]:
problem = Problem(
    'max',
    prompt,
   # fitness,
    solution_length = 2,
    #initial_bounds=([10,30],[20,40]),
    bounds=(
    [0,30],
    [50,500]
            ),
    dtype=torch.float32,
    #device='cpu'
)
problem


#1 sheath_gas = 30 #range? units?
#2 aux_gas = 10 #range? units?
#3 sweep_gas = 0,5 #units?
#4 spray_voltage = 3,5 #kV
#5 capillary_temperature = 275 #range? celcius
#6 capillary_voltage = 10 #range? volts 2-200
#7 tube_lens_voltage = 40 #range? volts 
#8 flow rate 5 - 50 ul/hr usually 15-20 
#Only positive ion mode 
#negative ion mode changes all the voltages to negative 
#ESI or APCI 
#APIC today more universial 

[2023-04-05 15:46:46] INFO     <  737> evotorch.core: Instance of `Problem` (id:140184366496400) -- The `dtype` for the problem's decision variables is set as torch.float32
[2023-04-05 15:46:46] INFO     <  737> evotorch.core: Instance of `Problem` (id:140184366496400) -- `eval_dtype` (the dtype of the fitnesses and evaluation data) is set as torch.float32
[2023-04-05 15:46:46] INFO     <  737> evotorch.core: Instance of `Problem` (id:140184366496400) -- The `device` of the problem is set as cpu
[2023-04-05 15:46:46] INFO     <  737> evotorch.core: Instance of `Problem` (id:140184366496400) -- The number of actors that will be allocated for parallelized evaluation is 0


In [32]:
def my_own_gaussian_mutation(x: torch.Tensor) -> torch.Tensor:
    # The default GaussianMutation of EvoTorch does not (yet) support different standard deviation values
    # per variable. However, we can define our own mutation operator which adds noise of different magnitudes
    # to different variables, like in this example:
    [_, solution_length] = x.shape
    dtype = x.dtype
    device = x.device

    # Generate Gaussian noise where each column has its own magnitude
    noise = (
        torch.randn(solution_length, dtype=dtype, device=device)
        * torch.tensor([3,40], dtype=dtype, device=device)
    )

    return x + noise

In [33]:
searcher = GeneticAlgorithm(
    problem,
    popsize=1,
    operators=[my_own_gaussian_mutation]
       #TwoPointCrossOver(problem, tournament_size=1)
   # ],
)

_ = StdOutLogger(searcher, interval=1)
count = 0
searcher.run(num_generations=10)


torch.Size([2])
1-Collision Energy, 2-Retention Time
Collision Energy  = ReadOnlyTensor(16.8534)
Retention Time = ReadOnlyTensor(293.1366)
torch.Size([2])
1-Collision Energy, 2-Retention Time
Collision Energy  = ReadOnlyTensor(14.8476)
Retention Time = ReadOnlyTensor(361.2993)
         iter : 1
    mean_eval : 40.0
pop_best_eval : 40.0
  median_eval : 40.0
    best_eval : 453456.0
   worst_eval : 2.0

torch.Size([2])
1-Collision Energy, 2-Retention Time
Collision Energy  = ReadOnlyTensor(16.8534)
Retention Time = ReadOnlyTensor(293.1366)
torch.Size([2])
1-Collision Energy, 2-Retention Time
Collision Energy  = ReadOnlyTensor(15.0528)
Retention Time = ReadOnlyTensor(289.3210)
         iter : 2
    mean_eval : 70.0
pop_best_eval : 70.0
  median_eval : 70.0
    best_eval : 453456.0
   worst_eval : 2.0

torch.Size([2])
1-Collision Energy, 2-Retention Time
Collision Energy  = ReadOnlyTensor(15.0528)
Retention Time = ReadOnlyTensor(289.3210)
torch.Size([2])
1-Collision Energy, 2-Retention Tim

In [31]:
list(searcher.population[:10])

[<Solution values=tensor([16.0248, 83.2338]), evals=tensor([456.])>]